In [2]:
import urllib2
import json
import numpy as np
import pylab
from bokeh.plotting import figure, output_file, show
from bokeh.palettes import Spectral11
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
# STEP 1: Create Database with prices 2000-2016
def pullDates():
    api_key='2B175EF8A024C543D84E2A581F139C0A'        
    url='http://api.eia.gov/series/?api_key='+ api_key+'&series_id=ELEC.PRICE.CA-ALL.M'
    json_obj= urllib2.urlopen(url)
    data = json.load(json_obj)
    dates=[]     
    for i in data['series'][0]['data']:
        dates.append(i[0])
    #statePrice=pd.DataFrame(np.nan)   
    return dates
dates=pullDates()
#print dates

In [4]:
def pullDataByState(state):
    api_key='2B175EF8A024C543D84E2A581F139C0A'        
    url='http://api.eia.gov/series/?api_key='+ api_key+'&series_id=ELEC.PRICE.'+str(state)+ '-ALL.M'
    json_obj= urllib2.urlopen(url)
    data = json.load(json_obj)
    prices=[]     
    for i in data['series'][0]['data']:
        prices.append(i[1])
    #statePrice=pd.DataFrame(np.nan)   
    return prices
#print pullDataByState('CA')

In [21]:
States=['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

def PriceDF():
    df=pd.DataFrame(np.nan, index=pullDates(), columns=States)    
    for state in States:
        df[state]=pullDataByState(state)
    return df
    
a=PriceDF()
b=a.transpose()
b.head()

In [6]:
#Left for when I attempt to make it dynamic
#a = a.reset_index()
#a=a.rename(columns={"index": "year_month"})
#a['Month']=[(str(i[:4])+'-'+str(i[4:])) for i in a['year_month']]

,201603,201602,201601,201512,201511,201510,201509,201508,201507,201506,...,200110,200109,200108,200107,200106,200105,200104,200103,200102,200101
AL,9.12,9.03,8.99,8.74,8.74,9.01,9.83,9.86,9.91,9.88,...,5.49,5.56,5.78,5.81,5.56,5.60,5.72,5.87,5.31,5.54
AK,17.92,17.64,17.50,17.49,18.21,18.00,18.10,18.35,18.73,18.56,...,11.19,10.24,10.78,10.39,10.58,10.55,10.59,10.91,9.92,9.97
AZ,9.45,9.44,9.27,9.20,9.23,10.31,11.07,11.22,11.43,11.30,...,7.71,7.77,7.87,7.78,8.01,7.86,6.87,6.57,6.48,6.35
AR,7.68,7.63,7.58,7.79,7.95,7.96,8.71,8.80,8.78,8.48,...,5.67,6.44,6.58,6.47,6.53,6.16,6.04,5.73,5.60,5.69
CA,14.34,14.53,14.75,14.48,15.11,15.70,17.45,17.29,17.40,16.20,...,12.74,13.13,13.12,13.47,11.34,9.82,9.88,9.89,9.63,8.71


In [23]:
#b.to_csv('./electricity_prices.csv', sep=',')

In [6]:
#STEP 2: Visualize the mean prices 2000-2016
stats=a.describe()

In [13]:
mean_price=stats.ix['mean']
    
#print mean_price.order(ascending=False)
#print mean_price['HI']
#print mean_price.describe()

Index([u'AL', u'AK', u'AZ', u'AR', u'CA', u'CO', u'CT', u'DE', u'FL', u'GA',
       u'HI', u'ID', u'IL', u'IN', u'IA', u'KS', u'KY', u'LA', u'ME', u'MD',
       u'MA', u'MI', u'MN', u'MS', u'MO', u'MT', u'NE', u'NV', u'NH', u'NJ',
       u'NM', u'NY', u'NC', u'ND', u'OH', u'OK', u'OR', u'PA', u'RI', u'SC',
       u'SD', u'TN', u'TX', u'UT', u'VT', u'VA', u'WA', u'WV', u'WI', u'WY'],
      dtype='object')

In [24]:
svg = open('usaLow.svg', 'r').read()
soup = BeautifulSoup(svg, 'html.parser')# check that 'html.parser' will work next tim -  , selfClosingTags=['defs','sodipodi:namedview'])
paths = soup.findAll('path')
colors=['#f1eef6', '#d7b5d8', '#df65b0', '#dd1c77', '#980043']
path_style = 'font-size:12px;fill-rule:nonzero;stroke:#FFFFFF;stroke-opacity:1;stroke-width:0.1;stroke-miterlimit:4;stroke-dasharray:none;stroke-linecap:butt;marker-start:none;stroke-linejoin:bevel;fill:'

In [25]:
for p in paths:
    try:
        rate = mean_price[p['id'][3:]]
    except KeyError:
        continue         
    
    if rate > 15:
        color_class = 4
    elif rate > 12:
        color_class = 3
    elif rate > 9:
        color_class = 2
    elif rate > 6:
        color_class = 1
    else:
        color_class = 0
    color = colors[color_class]
    p['style'] = path_style + color
    #p['fill'] = color

In [26]:
fp = open('mean_price.svg','wb')
fp.write(str(soup))
fp.close()